In [66]:
import time
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import ssl
import tqdm
import re
import json
import calendar
import datetime
import random
import bcolz
import numpy as np
import pickle
from ast import literal_eval
import uuid
import string
import nltk

## Helper functions

In [2]:
user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]

In [3]:
def get_soup(url):
    page = Request(url, headers = {"User-Agent": random.choice(user_agent_list)})
    html = urlopen(page, context = ctx).read()
    soup = BeautifulSoup(html, "html.parser")
    return soup

def get_companies(url):
    soup = get_soup(url).tbody
    tags = soup('tr')
    companies = [tag.find_all('a')[1].text for tag in tags]
    return companies

def prepend_zero(i):
    return '0'+ str(i) if i < 10 else i

def shorten_months(s):
    months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'sept', 'october', 'november', 'december']
    for month in months:
        s = s.replace(month, month[:3])
    return s

def get_dt(date_html):
    date_block = date_html.find(id='date')
    time_block = date_html.find(id='time')
    
    if date_block:
        date = date_block.text.lower().replace('.', '')
        date = shorten_months(date)
    else:
        regex_query = '(jan|feb|mar|apr|may|jun|jul|aug|sep|sept|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)\.* [0-9]*[0-9], [0-9][0-9][0-9][0-9]'
        date = re.search(regex_query, date_html.text.lower())
        if date is None:
            print('Could not find date!', date_html.text, '\n')
        else:
            date = date[0].replace('.','')
            splits = date.split(' ')
            if len(splits[1]) == 2:
                splits[1] = '0'+splits[1]
            date = ' '.join(splits)
            date = shorten_months(date)
            
    dt = None
    dt_time = None
    
    if time_block:
        time = time_block.text
        [time, ampm, time_zone] = time.split(' ')
        if len(time) == 4:
            time = '0'+time
        ampm = ampm.replace('.', '')
        dt_time = datetime.datetime.strptime(f'{time} {ampm}', "%I:%M %p").time()
    else:
        regex_query = '[0-9]*[0-9]:[0-9][0-9] (a\.*m\.*|p\.*m\.*)'
        time = re.search(regex_query, date_html.text.lower())
        if time is None:
            print('Could not find time!', date_html.text, '\n')
        else:
            time = time[0]
            time = time.replace('.', '')
            [time, ampm] = time.split(' ')
            if len(time) == 4:
                time = '0'+time
            dt_time = datetime.datetime.strptime(f'{time} {ampm}', "%I:%M %p").time()
            
    if date is not None:
        dt = datetime.datetime.strptime(date, '%b %d, %Y')
            
    if dt_time is None:
        return dt, dt_time
    
    mkt_open = datetime.time(hour = 9, minute = 30, second = 0)
    mkt_close = datetime.time(hour = 16, minute = 0, second = 0)
    
    after_close = dt_time > mkt_close
    
    return dt, after_close

## Create SSL Context

In [4]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

month_dict = {v: prepend_zero(k) for k,v in enumerate(calendar.month_abbr)}
full_dict = {}

## Get Transcripts

In [ ]:
companies = get_companies("http://www.slickcharts.com/sp500")

In [ ]:
with open('transcript_urls.json', 'r') as inp:
    transcript_urls = json.load(inp)

In [ ]:
companies_completed = []
for company, quarters in transcript_urls.items():
    for qtr, data in quarters.items():
        if 'transcript' in data:
            companies_completed.append(company)

In [ ]:
len(companies_completed)

In [ ]:
# transcript_urls = {}

In [ ]:
# Helper functions ctd.
def get_transcripts(companies, transcript_urls=None):
    if transcript_urls is None:
        transcript_urls = {}
#     for company in tqdm.tqdm_notebook(companies):
#         try:
#             url = f"https://www.fool.com/search/solr.aspx?q={company}%20earnings%20call%20transcript"
#             soup = get_soup(url)
#             result = soup.find_all("dl", class_ = "results")[0]
#             transcript_urls[company] = {}
#             for link in result.find_all('dt'):
#                 period = re.search('Q[0-9] [0-9][0-9][0-9][0-9]', link.text)
#                 if period is None:
#                     print(f'Skipping non-earnings call transcript url:{link.text}')
#                 else:
#                     url = link.a.get('href')
#                     transcript_urls[company][period[0]] = { 'url': url }
#         except:
#             print(f'Couldn\'t get transcripts for company {company}')
#             continue

    count = 0

    for company, urls in tqdm.tqdm_notebook(transcript_urls.items()):
        if company in companies_completed:
            continue
        quarters = list(urls.keys())
        np.random.shuffle(quarters)
        
        NUM_QUARTERS = 3
        quarters = quarters[:NUM_QUARTERS]
        
        for quarter in quarters:
            data = urls[quarter]
            soup = get_soup(data['url'])
            text = soup.find('span', class_ = 'article-content').find_all("p")
            early_lines = text[:3]
            dt_info = early_lines[0]
            for line in early_lines[1:]:
                dt_info.append(line)
            date, hours = get_dt(dt_info)

            try:
                mcap_soup = get_soup(f"http://www.zacks.com/stock/quote/{company}")
                for tr in mcap_soup.find_all("table", class_ = "abut_bottom"):
                    for td in tr.find_all("td"):
                        if td.text == "Market Cap":
                            mkt_cap = td.text + " " + td.find_next("td").text
                transcript_urls[company][quarter].update({
                    "transcript": str(soup),
                    "quarter": quarter,
                    "date": date.strftime('%b %d, %Y'),
                    "after_market": bool(hours) if hours is not None else None,
                    "market_cap": mkt_cap
                })
            except:
                print(f'Couldn\'t get stock information for {company} {quarter}')
        count += 1
        if count % 20 == 0:
            print('Saving transcripts...')
            with open('transcript_urls.json', 'w') as out:
                json.dump(transcript_urls, out, indent=2)
            print('Do you want to continue?')
            contin = input()
            if not contin:
                return
    return transcript_urls

In [ ]:
get_transcripts(companies, transcript_urls)
print('done')

## Assign unique identifiers to transcripts

In [ ]:
for company, quarters in transcript_urls.items():
    for quarter, data in quarters.items():
        if data.get('transcript') and data.get('date') and data.get('after_market') is not None:
            data['id'] = str(uuid.uuid4())

In [ ]:
with open('transcript_urls.json', 'w') as out:
    json.dump(transcript_urls, out, indent=2)

## Map ID to object

In [31]:
mapping = {}
for company, quarters in transcript_urls.items():
    for quarter, data in quarters.items():
        if 'id' in data:
            mapping[data['id']] = data

## Filter transcripts dictionary to only those that were collected

In [5]:
with open('transcript_urls.json', 'r') as inp:
    transcript_urls = json.load(inp)

valid_transcripts = []
for company, quarters in transcript_urls.items():
    for quarter, data in quarters.items():
        if data.get('transcript') and data.get('date') and data.get('after_market') is not None:
            valid_transcripts.append({
                'id': data['id'],
                'company': company,
                'quarter': quarter,
                'after_market': data['after_market'],
                'market_cap': data['market_cap'],
                'transcript': data['transcript'],
                'date': data['date'],
                'url': url
            })

## Add stock data objects

In [17]:
# Append stock info from days before/after earnings
# Don't run this on the entire company list, API token will fail out
# Split into 200/200/100 with API tokens:

# TODO: add in your own API tokens

tokens = [token1, token2, token3, token3, token5, sean_token]

for transcript in tqdm.tqdm_notebook(invalid_transcripts):
    if 'historical_info' in transcript:
        continue
    token = np.random.choice(tokens)
    base = "https://api.worldtradingdata.com/api/v1/history?"
    symbol = transcript['company']

    script_date = datetime.datetime.strptime(transcript['date'], '%b %d, %Y')

    change = datetime.timedelta(days = 1)
    day1 = script_date + change
    day2 = day1 + change
    day3 = day2 + change

    date_from = (script_date - datetime.timedelta(days = 10)).strftime("%Y-%m-%d")
    date_to = (script_date + datetime.timedelta(days = 3)).strftime("%Y-%m-%d")

    try:
        url = f"{base}symbol={symbol}&date_from={date_from}&date_to={date_to}&api_token={token}"
        response = urlopen(url)
        info = literal_eval(response.read().decode('utf-8'))

        if transcript['after_market'] == 0:
            # Script was reported before market hours so post-earnings day is current day
            next_day = script_date
        else:
            # Script was reported after market hours so post-earnings day is next day
            if day1.strftime("%Y-%m-%d") in info['history']:
                next_day = day1
            elif day2.strftime("%Y-%m-%d") in info['history']:
                next_day = day2
            else:
                next_day = day3

#         print(symbol)
        transcript["post_high_low"] = (info['history'][next_day.strftime("%Y-%m-%d")]['high'], 
                                               info['history'][next_day.strftime("%Y-%m-%d")]['low'])

        history_info = []
        days_recorded = 0
        num_days = 1
        while days_recorded < 5:
            days_before = next_day - datetime.timedelta(days = num_days)
            if days_before.strftime("%Y-%m-%d") in info['history']:
                history_info.append((days_before.strftime("%Y-%m-%d"), 
                                     (info['history'][days_before.strftime("%Y-%m-%d")]['high'],
                                      info['history'][days_before.strftime("%Y-%m-%d")]['low'])))
                days_recorded += 1
            num_days += 1
        transcript["historical_info"] = history_info
    except KeyError as e:
        print(e)
        continue

'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'
'history'


KeyboardInterrupt: 

## Filter to entries where the API worked

In [12]:
valid_transcripts_with_stock_data = []
invalid_transcripts = []
for transcript in valid_transcripts:
    if transcript.get('historical_info'):
        valid_transcripts_with_stock_data.append(transcript)
    else:
        invalid_transcripts.append(transcript)

In [18]:
len(valid_transcripts)

1492

In [11]:
len(valid_transcripts_with_stock_data)

909

## Save filtered data

In [36]:
with open('transcript_full_data.json', 'w') as out:
    json.dump(valid_transcripts_with_stock_data, out, indent=2)

## Fix Transcript Issue with new scrapes

In [35]:
for transcript in tqdm.tqdm_notebook(valid_transcripts_with_stock_data):
    url = mapping[transcript['id']]['url']
    transcript['url'] = url
    soup = get_soup(transcript['url'])
    transcript['transcript'] = str(soup)

In [96]:
with open('transcript_full_data.json', 'r') as inp:
    valid_transcripts_with_stock_data = json.load(inp)

## Tokenize and filter transcripts

In [97]:
for transcript in tqdm.tqdm_notebook(valid_transcripts_with_stock_data):
    html = transcript['transcript']
    soup = BeautifulSoup(html, 'html.parser')
    
    text = soup.find('span', class_ = 'article-content').find_all('p')
    text = list(filter(lambda x: not x.get('class') and not x.find_all('a'), text))
    new_text = []
    for p in reversed(text):
        if new_text:
            new_text.append(p)
            continue
        if p.text == '':
            continue
        if p.find_all('strong'):
            continue
        new_text.append(p)
    new_text.reverse()

    SPEAKER_START = '_SP_START_'
    SPEAKER_END = '_SP_END_'

    final_text = []
    for p in new_text:
        text_to_add = ''
        add_speaker = False
        if p.find_all('strong'):
            add_speaker = True
            text_to_add = p.text
        else:
            text_to_add = p.text
        text_to_add = text_to_add.lower()
        text_to_add = text_to_add.translate(str.maketrans('', '', string.punctuation))
        if add_speaker:
            text_to_add = ' '.join([SPEAKER_START, text_to_add, SPEAKER_END])
        text_to_add = nltk.word_tokenize(text_to_add)
        final_text.append(text_to_add)
        
    transcript['transcript'] = final_text

In [158]:
def mask_sentences(lines):
    new_lines = []
    for line in lines:
        new_tokens = [] 
        regexes = [
            '[0-9]+', 'million(s)*$', '(m)*m$', '[0-9]+(m)*m$', 
            '[0-9]+(b*)b$', 'trillion(s)*$', '[0-9]+t$', 'thousand(s)*$',
            'm$', '[0-9]+k$', 'hundred(s)*', 'ten(s)*', 'one', 'two', 'three', 'four',
            'five', 'six', 'seven', 'eight', 'nine', 'ten', 'twenty', 'thirty', 'fourty',
            'fifty', 'sixty', 'seventy', 'eighty', 'ninety'
        ]
        ignore = '|'.join(regexes)
        for token in line:
            if re.match(ignore, token):
                new_tokens.append('_MASKED_')
            else:
                new_tokens.append(token)
        new_lines.append(new_tokens)
    return new_lines

## Mask quantitative information

In [159]:
for transcript in tqdm.tqdm_notebook(valid_transcripts_with_stock_data):
    text = transcript['transcript']
    masked = mask_sentences(text)
    unrolled = []
    for line in masked:
        unrolled += line
    transcript['transcript'] = unrolled

In [160]:
with open('masked_full_transcripts.json', 'w') as out:
    json.dump(valid_transcripts_with_stock_data, out, indent=2)

## Transform strings to floats

In [163]:
for transcript in tqdm.tqdm_notebook(valid_transcripts_with_stock_data):
    transcript['post_high_low'] = tuple(map(float, transcript['post_high_low']))
    historical_info = list(map(lambda x: (x[0], tuple(x[1])), transcript['historical_info']))
    historical_info.reverse()
    transcript['historical_info'] = historical_info

In [167]:
for transcript in tqdm.tqdm_notebook(valid_transcripts_with_stock_data):
    market_cap = transcript['market_cap']
#     market_cap = re.search('[0-9]+\.[0-9]+', market_cap)[0]
    transcript['market_cap'] = float(market_cap)

In [168]:
with open('../../data/processed/masked_full_transcripts.json', 'w') as out:
    json.dump(valid_transcripts_with_stock_data, out, indent=2)